In [1]:
!pip install -r requirements.txt

In [16]:
!conda list bits

# packages in environment at /Users/josh/anaconda3/envs/genai:
#
# Name                    Version                   Build  Channel
bitsandbytes              0.42.0                   pypi_0    pypi


### Imports

In [4]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset

### Quantization

In [10]:
# https://huggingface.co/docs/peft/main/en/developer_guides/quantization
# import importlib
# print(importlib.metadata.version("bitsandbytes"))

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [11]:
# Load in the model
model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
tokenizer.padding_side = "right"

In [ ]:
model = prepare_model_for_kbit_training(quantized_model)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

### Datahandling

In [ ]:
from datasets import load_dataset

In [ ]:
train_dataset = load_dataset('json', data_files="/content/merged_math_problems_train.json", split="train")
val_dataset = load_dataset('json', data_files="/content/merged_math_problems_test.json")['train']

In [ ]:
train_dataset

Dataset({
    features: ['level', 'solution', 'type', 'problem'],
    num_rows: 7500
})

In [ ]:
val_dataset

5000

In [ ]:
text_format = """### Question: {problem_statement} [Answer] : {solution}"""

def formatting_prompts_func(dataset):
    output_texts = []

    for idx in range(len(dataset['level'])):
        problem = dataset['problem'][idx]
        solution = dataset['solution'][idx]
        text = text_format.format(problem_statement=problem, solution=solution)
        output_texts.append(text)

    return output_texts

In [ ]:
response_template = " [Answer] :"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key ` [Answer] :` in the following instance: <s> ### Question: The Cubs are playing the Red Sox in the World Series. To win the world series, a team must win 4 games before the other team does. If the Cubs win each game with probability $\dfrac{3}{5}$ and there are no ties, what is the probability that the Cubs will win the World Series? Express your answer as a percent rounded to the nearest whole percent. [Answer] : There are four possible cases for the Cubs winning the World Series, depending on the number of games that Red Sox win before the Cubs win their fourth game: the Red Sox can win no games, one game, two games, or three games. In general, if the Red Sox win exactly $k$ games before the Cubs win their 4th game, there will be a total of $3+k$ games played before the last one (which the Cubs must win), there will be a total of $\dbinom{3+k}{k}$ ways of selecting the games tha

KeyboardInterrupt: 